In [191]:
import pandas as pd
import numpy as np 
from datetime import datetime
import os 
import matplotlib.pyplot as plt
import seaborn as sns


In [192]:
#Créer un dossier pour les donées traitées
os.makedirs('data_processed', exist_ok=True)

In [193]:
#Charger les données
print("Chargement des données...")
frequentation_df= pd.read_parquet('data/frequentation_data.parquet')
parts_marche_df= pd.read_parquet('data/parts_marche_data.parquet')
allocine_df=pd.read_parquet('data/allocine_spider.parquet')

Chargement des données...


In [194]:
#Afficher les info sur les données chargées
print(f"Données de fréquentation : {frequentation_df.shape[0]} lignes, {frequentation_df.shape[1]} colonnes")
print(f"Données de parts de marché : {parts_marche_df.shape[0]} lignes, {parts_marche_df.shape[1]} colonnes")
print(f"Données de Allociné : {allocine_df.shape[0]} lignes, {allocine_df.shape[1]} colonnes")


Données de fréquentation : 179 lignes, 5 colonnes
Données de parts de marché : 38 lignes, 8 colonnes
Données de Allociné : 1908 lignes, 16 colonnes


In [195]:
#Nettoyage des données de fréquentation
print("\nNettoyage des données de fréquentation...")
#Convertir la colonne de date en datetime
# frequentation_df['Date Article'] = pd.to_datetime(frequentation_df['Date Article'], format= '%d %B %Y', errors='coerce')

mois_fr_to_en = {
    'janvier': 'January', 'février': 'February', 'mars': 'March', 
    'avril': 'April', 'mai': 'May', 'juin': 'June', 
    'juillet': 'July', 'août': 'August', 'septembre': 'September', 
    'octobre': 'October', 'novembre': 'November', 'décembre': 'December'
}

def convert_fr_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    
    for fr, en in mois_fr_to_en.items():
        date_str = date_str.replace(fr, en)
    
    return pd.to_datetime(date_str, format='%d %B %Y')
frequentation_df['Date Article'] = frequentation_df['Date Article'].apply(convert_fr_date)


Nettoyage des données de fréquentation...


In [196]:
#Créer des variables dérivées
frequentation_df['Année']= frequentation_df['Date Article'].dt.year
frequentation_df['Mois']=frequentation_df['Date Article'].dt.month
frequentation_df['Trimestre']=frequentation_df['Date Article'].dt.quarter

In [197]:
#Créer une variable pour la saison
frequentation_df['Saison'] = frequentation_df['Mois'].apply(
    lambda x: 'Hiver' if x in [12,1,2] else
    'Printemps' if x in [3,4,5] else
    'Eté' if x in [6,7,8] else 'Automne'
)

In [198]:
#Convertir les entrées en millions à des entrées en nombre absolu
frequentation_df['Entrées (millions)'] = frequentation_df['Année Courante']
frequentation_df['Entrées'] = frequentation_df['Année Courante'] * 1000000
frequentation_df['Entrées Année Précédente'] = frequentation_df['Année Précédente'] * 1000000

In [199]:
print("Données de fréquentation après nettoyage:")
print(frequentation_df.head())

Données de fréquentation après nettoyage:
  Date Article  Période  Année Courante  Année Précédente  Évolution (%)  \
0   2024-11-04  Janvier           13.71             14.82           -7.5   
1   2024-11-04  Février           15.07             18.03          -16.4   
2   2024-11-04     Mars           15.19             15.76           -3.6   
3   2024-11-04    Avril           11.89             18.61          -36.1   
4   2024-11-04      Mai           15.72             13.76           14.3   

   Année  Mois  Trimestre   Saison  Entrées (millions)     Entrées  \
0   2024    11          4  Automne               13.71  13710000.0   
1   2024    11          4  Automne               15.07  15070000.0   
2   2024    11          4  Automne               15.19  15190000.0   
3   2024    11          4  Automne               11.89  11890000.0   
4   2024    11          4  Automne               15.72  15720000.0   

   Entrées Année Précédente  
0                14820000.0  
1                180

In [200]:
#2. NETTOYAGE DES DONNEES DE PARTS DE MARCHE
print("\nNettoyage des données de parts de marché...")
#Convertir la colonne de date en datetime
mois_fr_to_en = {
    'janvier': 'January', 'février': 'February', 'mars': 'March', 
    'avril': 'April', 'mai': 'May', 'juin': 'June', 
    'juillet': 'July', 'août': 'August', 'septembre': 'September', 
    'octobre': 'October', 'novembre': 'November', 'décembre': 'December'
}

def convert_fr_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    
    for fr, en in mois_fr_to_en.items():
        date_str = date_str.replace(fr, en)
    
    return pd.to_datetime(date_str, format='%d %B %Y')
parts_marche_df['Date Article'] = parts_marche_df['Date Article'].apply(convert_fr_date)


Nettoyage des données de parts de marché...


In [201]:
# Créer des variables dérivées
parts_marche_df['Année'] = parts_marche_df['Date Article'].dt.year
parts_marche_df['Mois'] = parts_marche_df['Date Article'].dt.month
parts_marche_df['Trimestre'] = parts_marche_df['Date Article'].dt.quarter

In [202]:
print("Données de parts de marché après nettoyage:")
print(parts_marche_df.head())

Données de parts de marché après nettoyage:
  Date Article                                         Période  \
0   2024-11-04                               Dix premiers mois   
1   2024-11-04  Année glissante  (de novembre n-1 à octobre n)   
2   2023-12-01                              onze premiers mois   
3   2023-12-01  année glissante (de décembre n-1 à novembre n)   
4   2025-03-03       Année glissante (de mars n-1 à février n)   

   Films Français Année Courante  Films Français Année Précédente  \
0                           45.8                             39.0   
1                           45.7                             38.8   
2                           39.3                             43.2   
3                           38.0                             41.6   
4                           43.4                             40.0   

   Films Américains Année Courante  Films Américains Année Précédente  \
0                             36.9                               43.1  

In [203]:
# 3. NETTOYAGE DES DONNÉES ALLOCINE
print("\nNettoyage des données AlloCiné...")
# Convertir la colonne de date en datetime
allocine_df['date'] = pd.to_datetime(allocine_df['date'], errors='coerce')


Nettoyage des données AlloCiné...


In [204]:
# Créer des variables dérivées
allocine_df['Année_Sortie'] = allocine_df['date'].dt.year
allocine_df['Mois_Sortie'] = allocine_df['date'].dt.month
allocine_df['Trimestre_Sortie'] = allocine_df['date'].dt.quarter

In [205]:
# Créer une variable pour la saison de sortie
allocine_df['Saison_Sortie'] = allocine_df['Mois_Sortie'].apply(
    lambda x: 'Hiver' if x in [12, 1, 2] else
              'Printemps' if x in [3, 4, 5] else
              'Été' if x in [6, 7, 8] else 'Automne'
)

In [206]:
# Calculer le temps écoulé depuis la sortie (en jours)
today = pd.Timestamp.now().normalize()
allocine_df['Jours_Depuis_Sortie'] = (today - allocine_df['date']).dt.days

In [207]:
# Extraire les genres individuels (car ils sont séparés par des |)
allocine_df['genre_principal'] = allocine_df['genre'].str.split('|').str[0]

In [208]:
# Nettoyer les données de box-office
allocine_df['box_office_ratio'] = allocine_df['french_first_week_boxoffice'] / allocine_df['french_boxoffice']
allocine_df['box_office_ratio'] = allocine_df['box_office_ratio'].replace([np.inf, -np.inf], np.nan)

In [209]:
print("Données AlloCiné après nettoyage:")
print(allocine_df[['title', 'date', 'genre_principal', 'french_boxoffice', 'french_first_week_boxoffice', 'box_office_ratio']].head())

Données AlloCiné après nettoyage:
                title       date genre_principal  french_boxoffice  \
0       La Forteresse 2015-10-07        Aventure              1958   
1          38 témoins 2012-03-14           Drame            270549   
2  Keep the Lights On 2012-08-22           Drame             13267   
3       La Cage Dorée 2013-04-24         Comédie           1197974   
4    Zero Dark Thirty 2013-01-23          Action            524533   

   french_first_week_boxoffice  box_office_ratio  
0                          735          0.375383  
1                       104634          0.386747  
2                         5908          0.445315  
3                       288750          0.241032  
4                       237214          0.452238  


In [210]:
#4. ANALYSE EXPLORATOIRE DES DONNEES

#4.1 Analyse des films par genre
print("\nAnalyse des films par genre...")
genre_counts= allocine_df['genre_principal'].value_counts().reset_index()
genre_counts.columns= ['Genre', 'Nombre de films']
print(genre_counts.head(10))


Analyse des films par genre...
                Genre  Nombre de films
0               Drame              929
1             Comédie              271
2           Animation              195
3  Comédie dramatique              112
4            Aventure              107
5              Biopic               83
6              Action               81
7            Policier               51
8            Thriller               21
9   Epouvante-horreur               19


In [211]:
plt.figure(figsize=(12,6))
sns.barplot(data=genre_counts.head(10), x='Nombre de films', y='Genre')
plt.title('Nombre de films par genre principal')
plt.tight_layout()
plt.savefig('visualisations/film_par_genre.png')
plt.close()

In [212]:
import matplotlib.pyplot as plt
import numpy as np

# Création d'un exemple de données similaire
np.random.seed(42)
n_points = 100
critics_score = np.random.normal(3.5, 0.8, n_points)
viewers_score = critics_score + np.random.normal(0, 0.5, n_points)

# Création du graphique
plt.figure(figsize=(12, 8))

# Ligne de référence y=x (quand les scores sont égaux)
plt.plot([0, 5], [0, 5], '--', color='gray', alpha=0.5, label='Scores égaux')

# Calcul de la différence pour la coloration
difference = viewers_score - critics_score

# Création du scatter plot avec une colormap basée sur la différence
scatter = plt.scatter(critics_score, viewers_score, 
                     c=difference,  # Coloration basée sur la différence
                     cmap='RdYlBu',  # Rouge pour négatif (critiques > spectateurs), Bleu pour positif (spectateurs > critiques)
                     alpha=0.6,
                     s=100)  # Taille des points

# Ajout d'une barre de couleur
cbar = plt.colorbar(scatter)
cbar.set_label('Différence (Spectateurs - Critiques)', rotation=270, labelpad=15)

# Personnalisation
plt.xlabel('Score des critiques')
plt.ylabel('Score des spectateurs')
plt.title('Comparaison des scores critiques et spectateurs\nColoré selon la différence d\'opinion', pad=20)
plt.grid(True, linestyle='--', alpha=0.7)

# Limites des axes
plt.xlim(0, 5)
plt.ylim(0, 5)

# Annotations explicatives
plt.text(4.5, 1.5, 'Préféré par\nles critiques', color='red', ha='right')
plt.text(1.5, 4.5, 'Préféré par\nles spectateurs', color='blue', ha='left')

plt.tight_layout()

# Sauvegarde dans le dossier visualisations
plt.savefig('visualisations/scores_critiques_vs_spectateurs.png', dpi=300, bbox_inches='tight')
plt.close()

print("Visualisation sauvegardée dans 'visualisations/scores_critiques_vs_spectateurs.png'")

# Colormap utilisée : RdYlBu (Rouge-Jaune-Bleu)
# - Rouge vif : Les critiques ont beaucoup mieux noté que les spectateurs.
# - Rouge clair : Les critiques ont légèrement mieux noté que les spectateurs.
# - Jaune : Consensus entre critiques et spectateurs (différence proche de 0).
# - Bleu clair : Les spectateurs ont légèrement mieux noté que les critiques.
# - Bleu vif : Les spectateurs ont beaucoup mieux noté que les critiques.
# Transparence (alpha=0.6) : Permet de visualiser les zones de forte densité.

Visualisation sauvegardée dans 'visualisations/scores_critiques_vs_spectateurs.png'


In [213]:
# Calculer la corrélation
corr = allocine_df['critics_score'].corr(allocine_df['viewers_score'])
print(f"Corrélation entre scores critiques et spectateurs: {corr:.2f}")

Corrélation entre scores critiques et spectateurs: 0.28


In [214]:
# 4.3 Analyse du box-office par genre
print("\nAnalyse du box-office par genre...")
boxoffice_by_genre = allocine_df.groupby('genre_principal')['french_boxoffice'].agg(['mean', 'median', 'count']).reset_index()
boxoffice_by_genre = boxoffice_by_genre.sort_values('mean', ascending=False)
boxoffice_by_genre.columns = ['Genre', 'Moyenne', 'Médiane', 'Nombre de films']
print(boxoffice_by_genre.head(10))

plt.figure(figsize=(12, 6))
sns.barplot(data=boxoffice_by_genre.head(10), x='Moyenne', y='Genre')
plt.title('Box-office moyen par genre principal')
plt.xlabel('Box-office moyen (entrées)')
plt.tight_layout()
plt.savefig('visualisations/boxoffice_par_genre.png')
plt.close()


Analyse du box-office par genre...
               Genre       Moyenne    Médiane  Nombre de films
0             Action  1.826665e+06   840850.0               81
21           Western  1.438719e+06    55028.0                3
7   Comédie musicale  1.405050e+06  1405050.5                2
2           Aventure  1.325514e+06   448337.0              107
19   Science Fiction  1.301448e+06   866769.0                7
3             Biopic  7.654050e+05   216262.0               83
14       Fantastique  5.546627e+05    40587.0                7
5            Comédie  4.988811e+05   193551.0              271
20          Thriller  4.928897e+05   164330.0               21
17          Policier  4.147760e+05   198398.0               51


In [215]:
# 4.4 Analyse de la saisonnalité des sorties
print("\nAnalyse de la saisonnalité des sorties...")
season_counts = allocine_df['Saison_Sortie'].value_counts().reset_index()
season_counts.columns = ['Saison', 'Nombre de films']

# Réordonner les saisons
season_order = ['Hiver', 'Printemps', 'Été', 'Automne']
season_counts['Saison'] = pd.Categorical(season_counts['Saison'], categories=season_order, ordered=True)
season_counts = season_counts.sort_values('Saison')

print(season_counts)

plt.figure(figsize=(10, 6))
sns.barplot(data=season_counts, x='Saison', y='Nombre de films')
plt.title('Nombre de sorties de films par saison')
plt.tight_layout()
plt.savefig('visualisations/sorties_par_saison.png')
plt.close()


Analyse de la saisonnalité des sorties...
      Saison  Nombre de films
0      Hiver              532
2  Printemps              453
3        Été              394
1    Automne              529


In [216]:
# 4.5 Analyse du ratio première semaine / box-office total
print("\nAnalyse du ratio première semaine / box-office total...")

# Filtrer les valeurs aberrantes (garder uniquement les ratios entre 0 et 1)
ratio_data = allocine_df[(allocine_df['box_office_ratio'] <= 1) & (allocine_df['box_office_ratio'] > 0)]

# Calcul des statistiques principales
ratio_moyen = ratio_data['box_office_ratio'].mean()
ratio_median = ratio_data['box_office_ratio'].median()

# Création de la visualisation
plt.figure(figsize=(12, 7))

# Histogramme avec courbe de densité
sns.histplot(data=ratio_data, x='box_office_ratio', bins=20, kde=True, color='skyblue', alpha=0.6)

# Ajout des lignes verticales pour la moyenne et la médiane
plt.axvline(ratio_moyen, color='red', linestyle='--', label=f'Moyenne: {ratio_moyen:.2%}')
plt.axvline(ratio_median, color='green', linestyle='--', label=f'Médiane: {ratio_median:.2%}')

# Personnalisation du graphique
plt.title('Distribution du ratio Box-office première semaine / Box-office total', pad=20)
plt.xlabel('Ratio (Première semaine / Total)')
plt.ylabel('Nombre de films')

# Amélioration de la grille
plt.grid(True, linestyle='--', alpha=0.3)

# Amélioration de la légende
plt.legend(loc='upper right')

# Ajustement des axes pour plus de clarté
plt.xlim(0, 1)
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: '{:.0%}'.format(x)))

plt.tight_layout()

# Sauvegarde
plt.savefig('visualisations/ratio_premiere_semaine.png', dpi=300, bbox_inches='tight')
plt.close()

# Affichage des statistiques détaillées
print("\nStatistiques du ratio première semaine / box-office total:")
print(f"- Moyenne: {ratio_moyen:.1%} du box-office total")
print(f"- Médiane: {ratio_median:.1%} du box-office total")
print(f"- Nombre de films analysés: {len(ratio_data)}")
print(f"- Minimum: {ratio_data['box_office_ratio'].min():.1%}")
print(f"- Maximum: {ratio_data['box_office_ratio'].max():.1%}")

# Interprétation des quartiles
q1 = ratio_data['box_office_ratio'].quantile(0.25)
q3 = ratio_data['box_office_ratio'].quantile(0.75)
print(f"\nAnalyse par quartiles:")
print(f"- 25% des films font moins de {q1:.1%} de leur box-office total en première semaine")
print(f"- 50% des films font moins de {ratio_median:.1%} de leur box-office total en première semaine")
print(f"- 75% des films font moins de {q3:.1%} de leur box-office total en première semaine")
# Cette analyse examine le ratio entre les recettes de la première semaine et le box-office total d'un film.
# Elle permet de comprendre l'importance de la première semaine dans le succès commercial d'un film.
# Les statistiques (moyenne, médiane, quartiles) montrent si les films réalisent une grande partie de leurs recettes
# dès la première semaine ou s'ils ont une performance plus stable sur le long terme.
# Cela aide à identifier les films à succès rapide et ceux qui attirent un public sur une période prolongée.



Analyse du ratio première semaine / box-office total...

Statistiques du ratio première semaine / box-office total:
- Moyenne: 40.2% du box-office total
- Médiane: 39.2% du box-office total
- Nombre de films analysés: 1906
- Minimum: 0.0%
- Maximum: 100.0%

Analyse par quartiles:
- 25% des films font moins de 30.4% de leur box-office total en première semaine
- 50% des films font moins de 39.2% de leur box-office total en première semaine
- 75% des films font moins de 49.2% de leur box-office total en première semaine


In [217]:
# 5. ENREGISTRER LES DONNÉES NETTOYÉES
print("\nEnregistrement des données nettoyées...")
frequentation_df.to_parquet('data_processed/frequentation_cleaned.parquet')
parts_marche_df.to_parquet('data_processed/parts_marche_cleaned.parquet')
allocine_df.to_parquet('data_processed/allocine_cleaned.parquet')

print("Nettoyage et analyse exploratoire terminés!")


Enregistrement des données nettoyées...
Nettoyage et analyse exploratoire terminés!
